In [ ]:
pip install langchain

In [ ]:
pip install langchain_openai

In [ ]:
pip install beautifulsoup4


In [ ]:
pip install -U duckduckgo-search

In [ ]:
import os

OPENAI_API_KEY = "sk-" #PUT YOUR API KEY HERE


In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

QUERY_PROMPT = PromptTemplate(
    input_variables=["topic"],
    template ="""You are an AI language model assistant. Your task is to generate five different
    points of investigation of the given topic of the video. This will be used to search in Google.
    By generating multiple perspectives of a research, your goal is to help the user to have a complete essay of a topic
    Provide these investigation branches separates by newlines and starting with .:

    Topic of essay: {topic}

    DO NOT WRITE NUMBERS FOR LISTING
    """
)


llm = ChatOpenAI(temperature=.2, api_key= OPENAI_API_KEY)

llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT)
GENERAL_TOPIC = "SQL"
investigation_lines = llm_chain.invoke(GENERAL_TOPIC)
investigation_lines = investigation_lines["text"].strip().split("\n")
print(investigation_lines)





In [ ]:
investigation_lines = [topic.replace(". ", "") for topic in investigation_lines]

print(investigation_lines)

In [ ]:
from types import WrapperDescriptorType
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import re

search = DuckDuckGoSearchResults()

investigation_urls = {}

for topic in investigation_lines:
  result = search.run(topic)
  result = re.findall(r'link:\s*([^,\]]+)', result)
  investigation_urls[topic] = result

print(investigation_urls)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

content_dict = {}

for key, value in investigation_urls.items():
  content_list = []
  counter = 0
  for website in value:
    loader = WebBaseLoader(website)
    documents = loader.load()
    text = re.sub(r'\n+', '\n', documents[0].page_content)
    lines = text.split("\n")
    filtered_lines = [line for line in lines if len(line.split()) >= 8]
    text = "\n".join(filtered_lines)
    content_list.append(text)
    counter += 1
    if counter == 3:
      break

  content_dict[key] = content_list

print(content_dict)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(api_key= OPENAI_API_KEY, model= 'gpt-3.5-turbo-0125', verbose= True)

explanation_dict = {}

for key, value in content_dict.items():
  prompt = f"""
  Make an explanation of this topic with the given context:
  {topic}

  Context:
  {value}

  Do it as specific as posible, it is for an essay
  """
  response = llm.invoke(prompt)
  explanation_dict[key] = response.content

print(explanation_dict)

In [ ]:
for key, value in explanation_dict.items():
  print(key,": ", value)

In [ ]:
print(len(explanation_dict))

In [ ]:
script_llm = ChatOpenAI(model= 'gpt-4-0125-preview', temperature= .4, api_key= OPENAI_API_KEY)

script_prompt = """
You are a very useful AI whose only role is to take the given information and create a section of a Youtube Video.
Then, this sections will be united to create an entire video.
This information is not complete, this is the {iteration} iteration of {total_iterations}.
This is the information:
{information}

This is the topic of this section: {current_topic}
This is the next topic of the next section: {next_topic}
This is the general topic of the video: {topic}

Make this script in SPANISH
Call the person who is making the video: CONTENT CREATOR

DO NOT CREATE AN INTRO AND AN OUTRO
"""

template = PromptTemplate(
    input_variables=["current_topic", "next_topic","iteration", "total_iteration", "information", "topic"],
    template= script_prompt
)

script_chain = LLMChain(llm= script_llm, prompt= template)

script = []
counter = 1
# for key, value in explanation_dict.items():
#   response = script_chain.invoke({'iteration': counter,
#                                   'total_iterations': len(explanation_dict),
#                                   'information': value,
#                                   'current_topic': key
#                                   ''
#                                   'topic': key,
#                                   'output': ''})

# iterator = iter(explanation_dict.items())




keys = list(explanation_dict.keys())
num_keys = len(keys)

for i, key in enumerate(keys):
    next_key = keys[i + 1] if i < num_keys - 1 else None
    value = explanation_dict[key]

    response = script_chain.invoke({
        'iteration': counter,
        'total_iterations': num_keys,
        'information': value,
        'current_topic': key,
        'next_topic': next_key,
        'topic': GENERAL_TOPIC,
        'output': ''
    })
    script.append(response["text"])

In [ ]:
parser = ""
counter = 1
for topic in script:
    parser += "SECTION" + str(counter) + "\n" + topic
    counter += 1
print(parser)


In [ ]:
print(content_dict.keys())

In [ ]:
keys = list(explanation_dict.keys())
num_keys = len(keys)
for i, key in enumerate(keys):
    next_key = keys[i + 1] if i < num_keys - 1 else None
    value = explanation_dict[key]
    print(key, next_key)
